In [22]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import tqdm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import boosting_data
import config
import data
import wandb
from nets import ResNext, Densenet161
import ttach as tta

In [23]:
import ISBI_data
model = Densenet161()
ckpt = torch.load('boosting/densenet161_224x224/checkpoints/ISBI-densenet161-224x224-b3-epoch=020-val_loss=0.0791.ckpt', map_location=torch.device('cpu'))
new_dict = {k.replace('vit.', 'model.'): v for k, v in ckpt['state_dict'].items()}
model.load_state_dict(new_dict)
model.eval()
model.cuda()
model = tta.ClassificationTTAWrapper(model, tta.aliases.flip_transform())

testing_img_path = '../Evaluation_Set/Validation'
testing_df = '../Evaluation_Set/RFMiD_Validation_Labels.csv'
valset = boosting_data.ISBIDataset(testing_df, testing_img_path, testing=True, weight_csv=None, input_size=224)
N = len(valset)
batch_size = 2
dataloader = DataLoader(valset, batch_size=batch_size, shuffle=False, 
                        num_workers=24)

outs = np.zeros((N, 29))
labels = np.zeros((N, 29))
for i, (imgs, label, w) in enumerate(tqdm.tqdm(dataloader)):

    idx = i * batch_size
    imgs = imgs.cuda()
    out = model(imgs).detach().cpu().numpy()
    #out = np.round(out).astype('int').clip(1, None)
    outs[idx:idx + len(out),:] = out
    labels[idx:idx + len(label),:]  = label.detach().cpu().numpy()
sig = torch.nn.Sigmoid()
weight = np.zeros((29,))
count = np.zeros((29,))
rounded_valid_pred = np.round(sig(torch.tensor(outs)).numpy()).astype('int')
for i in range(labels.shape[0]):
    for j in range(labels.shape[1]):
        if labels[i][j] != rounded_valid_pred[i][j]:
            weight[j]+=1
        if labels[i][j] == 1:
            count[j] += 1
print(weight)
weight[weight==0]=1
weight = weight/count
weight = weight/min(weight)
weight_df = pd.DataFrame(weight)
weight_df.to_csv(f'boosting/densenet161_224x224/densenet_b4.csv', index=False)
print(weight)
print(count)

100%|██████████| 320/320 [00:51<00:00,  6.27it/s]

[65. 48. 24. 38. 41. 13. 11. 45.  7.  7.  5. 11. 56.  4.  2.  2. 24. 12.
  4.  3.  2.  4.  4.  9.  8.  7.  3.  2. 22.]
[ 1.          2.83076923  4.91659919  2.90015083  6.93846154  2.97647059
  3.72307692  5.38934911  7.78461538  3.20542986  7.78461538  7.78461538
  6.05470085  3.89230769  7.78461538  3.89230769  7.18579882  4.44835165
  7.78461538  4.67076923  7.78461538  5.18974359  2.22417582  6.36923077
 12.45538462  9.08205128  7.78461538  7.78461538  8.15531136]
[506. 132.  38. 102.  46.  34.  23.  65.   7.  17.   5.  11.  72.   8.
   2.   4.  26.  21.   4.   5.   2.   6.  14.  11.   5.   6.   3.   2.
  21.]


In [21]:
labels[639]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [10]:
a = np.array([63., 40., 25., 37., 43., 13., 11., 42.,  7.])
b = np.array([506., 132.,  38., 102.,  46.,  34.,  23.,  65., 7.])
c = a/b
c = c/min(c)
c

array([1.        , 2.43386243, 5.28404344, 2.9134765 , 7.50793651,
       3.07096172, 3.84126984, 5.18974359, 8.03174603])

In [8]:
weight_df = pd.DataFrame(weight)
weight_df.to_csv(f'boosting/weight_resnext_732_b0.csv', index=False)
print(weight)
print(count)

[ 1.          2.49747475  5.64912281  3.08169935  6.83333333  3.83333333
  3.33333333  4.24615385  7.66666667  4.50980392  7.66666667  6.96969697
  4.68518519  2.875      11.5         3.83333333  7.37179487  4.38095238
  7.66666667  7.66666667  7.66666667  3.83333333  2.19047619  6.27272727
 12.26666667  8.94444444  7.66666667  7.66666667  7.66666667]
[506. 132.  38. 102.  46.  34.  23.  65.   7.  17.   5.  11.  72.   8.
   2.   4.  26.  21.   4.   5.   2.   6.  14.  11.   5.   6.   3.   2.
  21.]
